In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -q "/content/drive/MyDrive/manufacturing_bigdata/05.project01/02.outlier_classifier/outlier_train.zip" -d "/content/train"

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import cv2
from google.colab.patches import cv2_imshow

import imgaug.augmenters as iaa
import imgaug as ia

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import albumentations as A
from collections import Counter
from keras import Input
from keras.models import Model
from keras import layers, models
from keras import optimizers, initializers, regularizers, metrics
from tensorflow.keras import optimizers
from glob import glob
from PIL import Image

In [ ]:
#이미지 경로

img_path = '/content/train/train/'

In [ ]:
# 이미지 크기

IM_HEIGHT = 224
IM_WIDTH  = 224

In [ ]:
# numpy 배열 출력시 가로로 길게 출력 되도록 설정

np.set_printoptions(linewidth=100000)

In [ ]:
# 데이터 프레임 로드

df = pd.read_csv(
    "/content/drive/MyDrive/manufacturing_bigdata/05.project01/02.outlier_classifier/train_df.csv",
    index_col = 'index')

In [ ]:
df

,file_name,class,state,label
index,,,,
0,10000.png,transistor,good,transistor-good
1,10001.png,capsule,good,capsule-good
2,10002.png,transistor,good,transistor-good
3,10003.png,wood,good,wood-good
4,10004.png,bottle,good,bottle-good
...,...,...,...,...
4272,14272.png,transistor,good,transistor-good
4273,14273.png,transistor,good,transistor-good
4274,14274.png,grid,good,grid-good


In [ ]:
# df["label"].unique() : 중복을 제거하고 리턴
label_list = sorted(df['label'].unique())

# label을 index 배열로 변환(클래스 분류를 하기 위해 index 마다 번호 부여)
df['label_int'] = df['label']

# 인덱스는 num, 레이블은 label에 저장
# label_int 컬럼 레이블을 숫자로 변환
for num, label in enumerate(label_list):
    df["label_int"].replace(label, num, inplace=True)

In [ ]:
# label_int 컬럼의 빈도수 조회
# sort_index() : 인덱스로 정렬

df["label_int"].value_counts().sort_index()

0      10
1      11
2      11
3     209
4       7
     ... 
83      8
84    240
85      9
86      9
87      8
Name: label_int, Length: 88, dtype: int64

### 학습

In [ ]:
import cv2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications.vgg16 import preprocess_input

In [ ]:
# VGG16 모델

model = VGG16()

553476096/553467096 [==============================] - 3s 0us/step


In [ ]:
df["label"]

index
0       transistor-good
1          capsule-good
2       transistor-good
3             wood-good
4           bottle-good
             ...       
4272    transistor-good
4273    transistor-good
4274          grid-good
4275        zipper-good
4276         screw-good
Name: label, Length: 4277, dtype: object

In [ ]:
df["label"].unique()

array(['transistor-good', 'capsule-good', 'wood-good', 'bottle-good', 'screw-good', 'cable-bent_wire', 'carpet-hole', 'hazelnut-good', 'pill-pill_type', 'cable-good', 'metal_nut-scratch', 'pill-good', 'screw-thread_side', 'zipper-fabric_border', 'leather-good', 'pill-scratch', 'toothbrush-good', 'hazelnut-crack', 'screw-manipulated_front', 'zipper-good', 'tile-good', 'carpet-good', 'metal_nut-good', 'bottle-contamination', 'grid-good', 'zipper-split_teeth', 'pill-crack', 'wood-combined', 'pill-color', 'screw-thread_top', 'cable-missing_cable', 'capsule-squeeze', 'zipper-rough', 'capsule-crack', 'capsule-poke', 'metal_nut-flip', 'carpet-metal_contamination', 'metal_nut-color', 'transistor-bent_lead', 'zipper-fabric_interior', 'leather-fold', 'tile-glue_strip', 'screw-scratch_neck', 'screw-scratch_head', 'hazelnut-cut', 'bottle-broken_large', 'bottle-broken_small', 'leather-cut', 'cable-cut_outer_insulation', 'zipper-squeezed_teeth', 'toothbrush-defective', 'cable-cut_inner_insulation', 

In [ ]:
categories = df["label"].unique()

In [ ]:
nb_classes = len(categories)

In [ ]:
nb_classes

88

In [ ]:
# 이미지 크기 : 가로, 세로 224로 설정

IM_HEIGHT = 224
IM_WIDTH = 224

In [ ]:
import math

In [ ]:
# 이미지 전처리

# X = []
# y = []

# files = glob(img_path + "/*.png")
# print("files = ", files)
# print("="*100)

# for i, f in enumerate(files):
#     img = Image.open(f)
#     img = img.convert("RGB")
#     img = img.resize((IM_HEIGHT, IM_WIDTH))
#     data = np.asarray(img)
#     img.close
    
#     X.append(data)
#     y.append(label)

In [ ]:
# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(df['file_name'].tolist(),
                                                    df['label_int'].tolist(),
                                                    test_size = 0.1,
                                                    stratify = df['label_int'],
                                                    random_state = 152)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
ia.seed(1)

In [ ]:
image_size = 224
batch_size = 256

In [ ]:
#학습 이미지 증강 객체
train_seq = iaa.Sequential([
    iaa.Resize({"height": image_size, "width": image_size}),
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.Sometimes(
        0.5,
        # Blur each image with varying strength using
        # gaussian blur (sigma between 0 and 3.0),
        # average/uniform blur (kernel size between 2x2 and 7x7)
        # median blur (kernel size between 3x3 and 11x11).
        iaa.OneOf([
            iaa.GaussianBlur((0, 3.0)),
            iaa.AverageBlur(k=(2, 7)),
            iaa.MedianBlur(k=(3, 11)),
            ]),
    ),
    iaa.Sometimes(
        0.3,
        #Either drop randomly 1 to 10% of all pixels (i.e. set
        # them to black) or drop them on an image with 2-5% percent
        # of the original size, leading to large dropped
        # rectangles.
        iaa.OneOf([
            iaa.Dropout((0.01, 0.1), per_channel=0.5),
            iaa.CoarseDropout(
                (0.03, 0.15), size_percent=(0.02, 0.05),
                per_channel=0.2
            ),
        ]),
    ),
    # crop some of the images by 0-10% of their height/width
    iaa.Sometimes(0.3 ,iaa.Crop(percent=(0, 0.1))),
    
    # Strengthen or weaken the contrast in each image.
    iaa.LinearContrast((0.75, 1.5)),
    # Add gaussian noise.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True) # apply augmenters in random order

In [ ]:
#테스트 이미지는 이미지 크기만 수정하고 다른 증강은 하지 않음
test_seq = iaa.Sequential([
    iaa.Resize({"height": image_size, "width": image_size})
], random_order=True) # apply augmenters in random order

In [ ]:
# 이미지를 증강시켜서 배치 사이즈 만큼씩 리턴하는 클래스 구현

class  ImageSequence(tf.keras.utils.Sequence):
    # 객체 생성시 실행되는 함수
    # 매개변수
    # image_arr : 이미지가 저장된 배열
    # label_arr : 이미지의 종류가 저장된 배열
    # batch_size : 배치 사이즈 (한번에 리턴할 파일 개수)
    # seq : 이미지 증강 객체

    def __init__(self, img_data_list ,label_int_list, batch_size, seq, img_path, total):
        # 매개변수들을 속성에 저장
        self.img_data_list  = img_data_list 
        self.label_int_list = label_int_list
        self.batch_size = batch_size
        self.seq = seq
        self.img_path = img_path
        self.total = total
    
    # batch_size씩 이미지를 리턴했을때 전체 이미지를 리턴하려명 몆번 반복해야 하는지 리턴
    def __len__(self):
        
        # math.ceil : 소숫점 1자리 올림 예) 6.1 -> 7   6 -> 6   6.0 -> 6

        # total 이미지의 개수 / batch_size (한번에 리턴할 이미지 개수) 의 올림을 리턴     
        return math.ceil(self.total / self.batch_size)

    # 학습시 batch_size 씩 이미지를 리턴하는 함수로 텐서플로우에서 학습시 model 객체에서 자동으로 호출하는 함수
    # 매개변수 idx : 몆번째 batch 인지가 저장되는 매개변수 0부터 시작
    def __getitem__(self, idx):
        #리턴할 이미지를 저장할 리스트
        batch_img = []
        #리턴할 레이블을 저장할 리스트
        batch_label = []
        #idx에 저장된 몆번째 batch인지 저장된 숫자에 batch_size 곱함
        start_index = int(idx * self.batch_size)
        #idx에 저장된 몆번째 batch인지 저장된 숫자 + 1 에 batch_size 곱함
        end_index = int((idx +1)*self.batch_size )

        #start_index 부터 end_index 까지 반복
        for index in range(start_index , end_index):
            #print("index = ", index)
            
            if index < self.total :
                # #이미지 파일명 저장            
                file_name = self.img_data_list[index]
                # #이미지를 읽어서 img에 저장
                img = cv2.imread(self.img_path + file_name)
                
                # #이미지를 읽지 못했다면
                if img is None:
                    #다음 이미지로 넘어감
                    continue
                # #이미지의 색상을 RGB 타입으로 변경
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                #이미지를 batch_img에 추가
                batch_img.append(img)
                #이미지 레이블을 batch_label에 추가
                batch_label.append(self.label_int_list[index])
                
        generate_img = None
        # batch_img를 증강시켜서 generate_img에 저장
        generate_img = self.seq(images=batch_img)
        #이미지를 float32타입으로 변환하고 255로 나눠줌        
        generate_img = np.array(generate_img, dtype="float32") /255.0
         
        #INPUT_DIM (입력 레이블의 종류 88)
        # batch_label을 one hot encoding 함
        batch_label_arr = np.eye(88)[batch_label]
        #batch_label_arr을 float32 타입 변환
        batch_label_arr = np.array(batch_label_arr, dtype="float32")
        
        #이미지와 레이블 리턴
        return generate_img,batch_label_arr

In [ ]:
# ImageSequence : 객체 생성
# X_train : 이미지
# y_train : 이미지 라벨
# batch_size : 배치 사이즈
# train_seq : 이미지 증강 객체
#img_path: 이미지 경로
#len(X_train) : 이미지 개수
train_sequence = ImageSequence(
    X_train,
    y_train,
    batch_size, 
    train_seq,
    img_path,
    len(X_train))

In [ ]:
# ImageSequence : 객체 생성
# X_test : 이미지
# y_test : 이미지 라벨
# 32 : 배치 사이즈
# test_seq : 이미지 증강 객체 (테스트 데이터는 증강 하지 않고 이미지 크기만 설정)
#img_path: 이미지 경로
#len(X_test) : 이미지 개수
test_sequence = ImageSequence(
    X_test,
    y_test,
    batch_size, 
    test_seq, 
    img_path,
    len(X_test))

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
# vgg16 모델 불러오기
pre_trained_vgg = VGG16(weights = 'imagenet',
                        include_top = False,
                        input_shape = (224, 224, 3))
pre_trained_vgg.trainable = False
pre_trained_vgg.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [ ]:
#vgg16 밑에 레이어 추가
additional_model = models.Sequential()
additional_model.add(pre_trained_vgg)
additional_model.add(layers.Flatten())
additional_model.add(layers.Dense(512, #kernel_regularizer = regularizers.l1_l2(l1=0.001,l2=0.001),
                                  activation='relu'))
additional_model.add(layers.Dropout(0.5))
additional_model.add(layers.Dense(256, #kernel_regularizer = regularizers.l1_l2(l1=0.001,l2=0.001),
                                  activation='relu'))
additional_model.add(layers.Dropout(0.5))
# additional_model.add(layers.Dense(64, kernel_regularizer = regularizers.l1_l2(l1=0.001,l2=0.001),activation='relu'))
# additional_model.add(layers.Dropout(0.5))
additional_model.add(layers.Dense(88, activation='softmax'))

additional_model.compile(loss='categorical_crossentropy',
                         optimizer=optimizers.RMSprop(learning_rate=1e-4),
                         metrics=['acc'])

In [ ]:
from sklearn.utils import class_weight
class_weights = dict(
    zip(
        np.unique(y_train),
        class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
    )
)
#class_weights

In [ ]:
def scheduler(epoch: int, lr: float) -> float:
    if epoch != 0 and epoch % 15 == 0 and lr > 1e-6:
        return lr * 0.1
    # if epoch < 30:
    #     return 1e-3
    # elif epoch < 50:
    #     return 1e-4
    # elif epoch < 100:
    #     return 1e-5
    else:
        return lr
lr_scheduler_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

early_stopping = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 30, restore_best_weights = True)

checkpoint_filepath = "/content/drive/MyDrive/best_model.h5"
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_loss",
    save_best_only=True,
    vervose=1,
    save_freq="epoch"
)

In [ ]:
# 모델 학습
history = additional_model.fit(train_sequence,
                               epochs=100, 
                               validation_data=test_sequence,
                               class_weight = class_weights,
                               callbacks = [
                                            # early_stopping,
                                            checkpoint_callback,
                                            lr_scheduler_callback,
                                            ])

Epoch 1/100
16/16 [==============================] - 319s 20s/step - loss: 4.7822 - acc: 0.0182 - val_loss: 4.2143 - val_acc: 0.1425 - lr: 1.0000e-04
Epoch 2/100
16/16 [==============================] - 274s 17s/step - loss: 4.5217 - acc: 0.0268 - val_loss: 4.1751 - val_acc: 0.0771 - lr: 1.0000e-04
Epoch 3/100
16/16 [==============================] - 332s 21s/step - loss: 4.4678 - acc: 0.0369 - val_loss: 4.0962 - val_acc: 0.1589 - lr: 1.0000e-04
Epoch 4/100
16/16 [==============================] - 298s 19s/step - loss: 4.3962 - acc: 0.0460 - val_loss: 3.9739 - val_acc: 0.1963 - lr: 1.0000e-04
Epoch 5/100
16/16 [==============================] - 271s 16s/step - loss: 4.3740 - acc: 0.0546 - val_loss: 3.8249 - val_acc: 0.2757 - lr: 1.0000e-04
Epoch 6/100
16/16 [==============================] - 309s 20s/step - loss: 4.2723 - acc: 0.0751 - val_loss: 3.7461 - val_acc: 0.2640 - lr: 1.0000e-04
Epoch 7/100
16/16 [==============================] - 345s 22s/step - loss: 4.2523 - acc: 0.0831 - va

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()